In [1]:
pip install pandas requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from geopy.distance import geodesic

ModuleNotFoundError: No module named 'geopy'

In [4]:
import pandas as pd
from geopy.distance import geodesic

In [19]:
data = pd.read_csv('./OYO_PU.csv')

In [17]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [20]:

# Print column names to check for the exact names
print("Columns in the DataFrame:", data.columns)

Columns in the DataFrame: Index(['State', 'LGA', 'Ward', 'PU-Code', 'PU-Name', 'Full Address',
       'Latitude', 'Longitude', 'Accredited_Voters', 'Registered_Voters',
       'Results_Found', 'Transcription_Count', 'Result_Sheet_Stamped',
       'Result_Sheet_Corrected', 'Result_Sheet_Invalid',
       'Result_Sheet_Unclear', 'Result_Sheet_Unsigned', 'APC', 'LP', 'PDP',
       'NNPP', 'Results_File'],
      dtype='object')


In [21]:
# Ensure Latitude and Longitude columns exist and are named correctly
if 'Latitude' not in data.columns or 'Longitude' not in data.columns:
    raise KeyError("The DataFrame does not contain 'Latitude' and 'Longitude' columns. Please check the column names.")

In [22]:
# Convert Latitude and Longitude to numeric
data['Latitude'] = pd.to_numeric(data['Latitude'], errors='coerce')
data['Longitude'] = pd.to_numeric(data['Longitude'], errors='coerce')

In [23]:
# Define the radius in kilometers
radius_in_km = 1

In [24]:
# Function to calculate distance and identify neighbors
def find_neighbors(data, radius_in_km):
    neighbors = {}
    # Initialize tqdm for progress tracking
    for idx, row in tqdm(data.iterrows(), total=len(data), desc="Identifying neighbors"):
        current_unit = (row['Latitude'], row['Longitude'])
        neighbors[idx] = []
        for i, other_row in data.iterrows():
            if idx != i:
                other_unit = (other_row['Latitude'], other_row['Longitude'])
                if pd.notna(current_unit[0]) and pd.notna(current_unit[1]) and pd.notna(other_unit[0]) and pd.notna(
                        other_unit[1]):
                    distance = geodesic(current_unit, other_unit).km
                    if distance <= radius_in_km:
                        neighbors[idx].append(i)
    return neighbors

In [27]:
from tqdm import tqdm

# Identify neighbors
neighbors_dict = find_neighbors(data, radius_in_km)

Identifying neighbors: 100%|██████████| 3899/3899 [1:27:55<00:00,  1.35s/it]


In [30]:
# Convert neighbor indices to their corresponding names or IDs
data['Neighbors'] = data.index.map(lambda idx: ', '.join(data.loc[neighbors_dict[idx], 'PU-Name'].astype(str)))
# Save the updated dataframe to an Excel file
output_file_path = r'./file.xlsx'
data.to_excel(output_file_path, index=False)
print(f"Updated file saved as '{output_file_path}'")

Updated file saved as './file.xlsx'
